<a href="https://colab.research.google.com/github/RajasreePushpan/yes_no_questions/blob/main/yes_or_no_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -U

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("/content/True or False questions - Sheet1 (1).csv",on_bad_lines='skip', engine="python")
data.head()

,Number,Question,Potential Answer,Correct Answer,Ground Truth,Location,Type
0,1,Statistics involves the science of collecting ...,True/False,True,"Statistics is the science of collecting, organ...",Page 3,True/False
1,2,Organizing data is not a part of the statistic...,True/False,False,"Statistics is the science of collecting, organ...",Page 3,True/False
2,3,Presenting data is an important aspect of stat...,True/False,True,"Statistics is the science of collecting, organ...",Page 3,True/False
3,4,Analyzing data is not necessary in statistics.,True/False,False,"Statistics is the science of collecting, organ...",Page 3,True/False
4,5,"Descriptive statistics help in organizing, sum...",True/False,True,Descriptive statistics include methods of orga...,page 4,True/False


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import AlbertTokenizer, AlbertForSequenceClassification

In [5]:
# Convert the 'Correct Answer' column to string type, clean by stripping spaces, and convert to lowercase
data['Correct Answer'] = data['Correct Answer'].astype(str).str.strip().str.lower()

# Map 'true'/'false' to 1/0
data['Correct Answer'] = data['Correct Answer'].map({'true': 1, 'false': 0})

# Check for NaN values to debug
print(data['Correct Answer'].isna().sum())

0


In [6]:
# Save the modified dataframe to a new CSV file (optional)
data.to_csv("/content/True_or_False_questions_modified.csv", index=False)

In [7]:
data = data[['Question','Correct Answer']]
data = data[0:250]
data.head()

,Question,Correct Answer
0,Statistics involves the science of collecting ...,1
1,Organizing data is not a part of the statistic...,0
2,Presenting data is an important aspect of stat...,1
3,Analyzing data is not necessary in statistics.,0
4,"Descriptive statistics help in organizing, sum...",1


In [8]:
# Split data into train and validation sets
X = list(data["Question"])
y = list(data["Correct Answer"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [9]:
# Initialize ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)
model = model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Tokenize the data
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors='pt')
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [11]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [12]:
# Convert tokenized data to datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [13]:
# Define metrics computation
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [15]:
# Define training arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [17]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.759700,0.683708,0.560000,0.000000,0.000000,0.000000
2,0.666200,0.608569,0.720000,0.900000,0.409091,0.562500
3,0.560700,0.586647,0.660000,0.580645,0.818182,0.679245


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=75, training_loss=0.6657603931427002, metrics={'train_runtime': 8.847, 'train_samples_per_second': 67.819, 'train_steps_per_second': 8.477, 'total_flos': 1036206756000.0, 'train_loss': 0.6657603931427002, 'epoch': 3.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.5866469740867615,
 'eval_accuracy': 0.66,
 'eval_precision': 0.5806451612903226,
 'eval_recall': 0.8181818181818182,
 'eval_f1': 0.679245283018868,
 'eval_runtime': 0.2542,
 'eval_samples_per_second': 196.697,
 'eval_steps_per_second': 27.538,
 'epoch': 3.0}

In [19]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9455,  0.1287]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.2546, 0.7454]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.2546095 , 0.74539053]], dtype=float32)

In [20]:
trainer.save_model('albert_CustomModel')